### Extracting images of pieces from chessboards

In [3]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle

size = 160             #choosing the size of the image
box_size = int(size/8) #size of each cell

In [4]:
#order of pieces except pawn
pieces = ['rook', 'knight', 'bishop', 'queen', 'king', 'bishop', 'knight', 'rook']
 
#iterate over every chessboard
for file in os.listdir('./test_boards'):
    
    img = cv.imread('./test_boards/' + file)       #extract chessboard image
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)     #grayscale image
    gray = cv.resize(gray, (size, size))           #resizing the image

    #segragating each box image for classification
    for x in range(0, 8):
        for y in range(0, 8):
            #destination of cropped image
            directory = './test_cropped/'
            if x==0:
                directory = directory + 'b_' + pieces[y]
            elif x==1:
                directory = directory + 'b_pawn'
            elif x==6:
                directory = directory + 'w_pawn'
            elif x==7:
                directory = directory + 'w_' + pieces[y]
            else:
                directory = directory + 'empty'
            directory = directory + '/'
            
            #extract a single cell from the board
            crop = gray[x*box_size:x*box_size+box_size, y*box_size:y*box_size+box_size]
            
            #save the cell
            filename = directory + file + '_' + str(x) + 'x' + str(y) + '.jpg'
            if not os.path.exists(filename):
                cv.imwrite(filename, crop)

### Creating the test data

In [6]:
DATADIR = "./test_cropped"
CATEGORIES = ['b_rook', 'b_knight', 'b_queen', 'w_king', 'b_king', 'w_queen', 'w_bishop', 'w_knight', 'b_bishop', 'w_rook', 'empty', 'b_pawn', 'w_pawn']

test_data = []

def create_test_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv.imread(os.path.join(path,img), cv.IMREAD_GRAYSCALE)
            test_data.append([img_array, class_num])
            
create_test_data()        #create training data
random.shuffle(test_data) #shuffling data to improve accuracy


In [8]:
print(len(test_data))

448


In [10]:
X_test = []
y_test = []

for features, label in test_data:
    X_test.append(features)
    y_test.append(label)

#Converting the list X to numpy array as we cannot feed a list to neural network
X_test = np.array(X_test).reshape(-1, box_size, box_size, 1)
X_test = X_test/255.0

### Storing data in pickle

In [11]:
pickle_out = open("X_test.pickle", "wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open("y_test.pickle", "wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()